In [50]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.holtwinters import ExponentialSmoothing

import warnings

warnings.filterwarnings('ignore')
sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [55]:
egresos_torax = pd.read_csv("../data/processed/egresos_torax.csv", sep=";", encoding="latin-1")
dias_estada_grd = pd.read_excel("../data/processed/dias_estada_hospitalizados.xlsx")
canasta_hospitalizados_grd = pd.read_excel(
    "../data/processed/canasta_procedimientos_hospitalizados.xlsx"
)

Primero se quiere hacer un forecasting total de los egresos (sin desglose por diagnostico),
y tambien de los procedimientos necesarios totales

In [67]:
DIAGNOSTICO = "C341"

In [78]:
resumen_egresos_totales = egresos_torax.query("DIAG1 == @DIAGNOSTICO and ANO_EGRESO == 2019").groupby(["ANO_EGRESO"])["n_egresos"].sum().reset_index()
resumen_procedimientos_totales = (
    canasta_hospitalizados_grd.query("DIAGNOSTICO1 == 'C34.1' and ANIO_EGRESO == 2019"))

In [85]:
EGRESOS_AUMENTADOS = 300
EGRESOS_2019 = resumen_egresos_totales.n_egresos.iloc[0]
AUMENTO_PROPORCIONAL = EGRESOS_AUMENTADOS / EGRESOS_2019

In [93]:
procedimientos_aumentados = resumen_procedimientos_totales.copy()
COLS_A_AUMENTAR = ["cantidad_procedimientos", "cantidad_pacientes_distintos"]

procedimientos_aumentados[COLS_A_AUMENTAR] = (
    procedimientos_aumentados[COLS_A_AUMENTAR] * AUMENTO_PROPORCIONAL
)

In [94]:
resumen_procedimientos_totales

,ANIO_EGRESO,DIAGNOSTICO1,procedimiento,cantidad_procedimientos,cantidad_pacientes_distintos,cantidad_proced_por_pacientes,Código_x,Descripción_x,Categoría_x,Sección_x,Capítulo_x,Versión,Código_y,Descripción_y,Categoría_y,Sección_y,Capítulo_y
977,2019,C34.1,99.29,175,176,0.994318,99.29,INYECCION SUSTANCIA TERAPEUTICA O PROFILACTICA...,99 OTROS PROCEDIMIENTOS NO QUIRURGICOS,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,CIE-v2013,C34.1,"Tumor maligno del lóbulo superior, bronquio o ...",C34 TUMOR MALIGNO DE LOS BRONQUIOS Y DEL PULMÓN,C30-C39 NEOPLASIAS MALIGNAS DE ÓRGANOS RESPIR...,Cap.02 NEOPLASIAS (C00-D49)
978,2019,C34.1,90.59,169,176,0.960227,90.59,EXAMEN MICROSCOPICO SANGRE.OTRO,90 EXAMEN MICROSCOPICO I,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,CIE-v2013,C34.1,"Tumor maligno del lóbulo superior, bronquio o ...",C34 TUMOR MALIGNO DE LOS BRONQUIOS Y DEL PULMÓN,C30-C39 NEOPLASIAS MALIGNAS DE ÓRGANOS RESPIR...,Cap.02 NEOPLASIAS (C00-D49)
979,2019,C34.1,99.19,140,176,0.795455,99.19,INYECCION ANTICOAGULANTE,99 OTROS PROCEDIMIENTOS NO QUIRURGICOS,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,CIE-v2013,C34.1,"Tumor maligno del lóbulo superior, bronquio o ...",C34 TUMOR MALIGNO DE LOS BRONQUIOS Y DEL PULMÓN,C30-C39 NEOPLASIAS MALIGNAS DE ÓRGANOS RESPIR...,Cap.02 NEOPLASIAS (C00-D49)
980,2019,C34.1,93.96,131,176,0.744318,93.96,ENRIQUECIMIENTO OXIGENO OTRO,"93 FISIOTERAPIA, TERAPIA RESPIRATORIA,REHABILI...",16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,CIE-v2013,C34.1,"Tumor maligno del lóbulo superior, bronquio o ...",C34 TUMOR MALIGNO DE LOS BRONQUIOS Y DEL PULMÓN,C30-C39 NEOPLASIAS MALIGNAS DE ÓRGANOS RESPIR...,Cap.02 NEOPLASIAS (C00-D49)
981,2019,C34.1,87.44,128,176,0.727273,87.44,RADIOGRAFIA TORAX RUTINARIA,87 RADIOLOGIA DIAGNOSTICA,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,CIE-v2013,C34.1,"Tumor maligno del lóbulo superior, bronquio o ...",C34 TUMOR MALIGNO DE LOS BRONQUIOS Y DEL PULMÓN,C30-C39 NEOPLASIAS MALIGNAS DE ÓRGANOS RESPIR...,Cap.02 NEOPLASIAS (C00-D49)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160,2019,C34.1,33.78,1,176,0.005682,33.78,RETIRADA ENDOSCOPICA DISPOSITIVO O SUSTANCIA B...,33 OTRAS OPERACIONES SOBRE PULMON Y BRONQUIO,06. (30 34) OPERACIONES SOBRE EL APARATO RESPI...,06. (30 34) OPERACIONES SOBRE EL APARATO RESPI...,CIE-v2013,C34.1,"Tumor maligno del lóbulo superior, bronquio o ...",C34 TUMOR MALIGNO DE LOS BRONQUIOS Y DEL PULMÓN,C30-C39 NEOPLASIAS MALIGNAS DE ÓRGANOS RESPIR...,Cap.02 NEOPLASIAS (C00-D49)
1161,2019,C34.1,33.29,1,176,0.005682,33.29,PROCEDIMIENTO DIAGNOSTICO PULMON OTRO,33 OTRAS OPERACIONES SOBRE PULMON Y BRONQUIO,06. (30 34) OPERACIONES SOBRE EL APARATO RESPI...,06. (30 34) OPERACIONES SOBRE EL APARATO RESPI...,CIE-v2013,C34.1,"Tumor maligno del lóbulo superior, bronquio o ...",C34 TUMOR MALIGNO DE LOS BRONQUIOS Y DEL PULMÓN,C30-C39 NEOPLASIAS MALIGNAS DE ÓRGANOS RESPIR...,Cap.02 NEOPLASIAS (C00-D49)
1162,2019,C34.1,32.39,1,176,0.005682,32.39,RESECCION PULMON SEGMENTARIA.OTRA Y NEOM,32 EXCISION DE PULMON Y BRONQUIO,06. (30 34) OPERACIONES SOBRE EL APARATO RESPI...,06. (30 34) OPERACIONES SOBRE EL APARATO RESPI...,CIE-v2013,C34.1,"Tumor maligno del lóbulo superior, bronquio o ...",C34 TUMOR MALIGNO DE LOS BRONQUIOS Y DEL PULMÓN,C30-C39 NEOPLASIAS MALIGNAS DE ÓRGANOS RESPIR...,Cap.02 NEOPLASIAS (C00-D49)
1163,2019,C34.1,32.23,1,176,0.005682,32.23,ABLACION ABIERTA TEJIDO O LESION PULMON,32 EXCISION DE PULMON Y BRONQUIO,06. (30 34) OPERACIONES SOBRE EL APARATO RESPI...,06. (30 34) OPERACIONES SOBRE EL APARATO RESPI...,CIE-v2013,C34.1,"Tumor maligno del lóbulo superior, bronquio o ...",C34 TUMOR MALIGNO DE LOS BRONQUIOS Y DEL PULMÓN,C30-C39 NEOPLASIAS MALIGNAS DE ÓRGANOS RESPIR...,Cap.02 NEOPLAS